# Reshape Examples

In [1]:
# basic setup
import datetime
from dateutil.relativedelta import relativedelta
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
from pathlib import Path
import pytz
import statsmodels.api as sm
import statsmodels.formula.api as smf
import string
import sys



## Pivot Basic

In [2]:
# i is the index in the wide version
# j is the name of the second index created from suffixes in the long version
# stubnames are the variables that are being indexed i.e. the first part of the variables that are currently given across the second index
df = pd.DataFrame({'gdp1': [100, 102], 'gdp2': [100, 101], 'year': [2000, 2001]})
print(df)
df2 = pd.wide_to_long(df, stubnames = 'gdp', i = 'year', j = 'country')
df2 = df2.reset_index()
print(df2)
# unpivot
df3 = pd.pivot(df2, index = 'year', columns = 'country', values = 'gdp')
df3.columns = ['gdp' + str(gdpcol) for gdpcol in df3.columns]
df3 = df3.reset_index()
print(df3)

   gdp1  gdp2  year
0   100   100  2000
1   102   101  2001
   year  country  gdp
0  2000        1  100
1  2001        1  102
2  2000        2  100
3  2001        2  101
   year  gdp1  gdp2
0  2000   100   100
1  2001   102   101


## Pivot Multiple Variables

In [3]:
# with multiple variables
df = pd.DataFrame({'gdp1': [100, 102], 'gdp2': [100, 101], 'unemp1': [5, 4], 'unemp2': [4, 3], 'year': [2000, 2001]})
print(df)
df2 = pd.wide_to_long(df, stubnames = ['gdp', 'unemp'], i = 'year', j = 'country')
df2 = df2.reset_index()
print(df2)
# unpivot
df3 = pd.pivot(df2, index = 'year', columns = 'country', values = ['gdp', 'unemp'])
# remove multi-index
df3.columns = [tup[0] + str(tup[1]) for tup in df3.columns]
df3 = df3.reset_index()
print(df3)

   gdp1  gdp2  unemp1  unemp2  year
0   100   100       5       4  2000
1   102   101       4       3  2001
   year  country  gdp  unemp
0  2000        1  100      5
1  2001        1  102      4
2  2000        2  100      4
3  2001        2  101      3
   year  gdp1  gdp2  unemp1  unemp2
0  2000   100   100       5       4
1  2001   102   101       4       3


## Pivot - Suffix Not a Number

In [4]:
# need to specify if suffix is not a number using suffix = '\D+'
# suffix using numbers (only) (default): suffix = '\d+'
# suffix using letters (only): suffix = '\D+'
# suffix using letters or numbers: suffix = '\S+'
# adding separator
# adding additional index variable
df = pd.DataFrame({'gdp_usa': [100, 102], 'gdp_japan': [100, 101], 'unemp_usa': [5, 4], 'unemp_japan': [4, 3], 'year': [2000, 2001], 'worldgpd': [200, 203]})
print(df)
df2 = pd.wide_to_long(df, stubnames = ['gdp', 'unemp'], i = 'year', j = 'country', sep = '_', suffix = '\D+')
df2 = df2.reset_index()
print(df2)
# unpivot
df3 = pd.pivot(df2, index = 'year', columns = 'country', values = ['gdp', 'unemp'])
# with multiple values, get multi-index column that need to combine manually
df3.columns = [tup[0] + '_' + tup[1] for tup in df3.columns]
df3 = df3.reset_index()
print(df3)

   gdp_usa  gdp_japan  unemp_usa  unemp_japan  year  worldgpd
0      100        100          5            4  2000       200
1      102        101          4            3  2001       203
   year country  worldgpd  gdp  unemp
0  2000     usa       200  100      5
1  2001     usa       203  102      4
2  2000   japan       200  100      4
3  2001   japan       203  101      3
   year  gdp_japan  gdp_usa  unemp_japan  unemp_usa
0  2000        100      100            4          5
1  2001        101      102            3          4


## Pivot - Multiple Index

In [5]:
# i is the index in the wide version
# j is the name of the second index created from suffixes in the long version
# stubnames are the variables that are being indexed i.e. the first part of the variables that are currently given across the second index
df = pd.DataFrame({'source': ['AUS', 'CAN', 'AUS', 'CAN'], 'dest': ['AUS', 'AUS', 'CAN', 'CAN'], 'exports_2000': [1, 2, 3, 4], 'exports_2001': [2, 3, 4, 5]})
print(df)
df2 = pd.wide_to_long(df, stubnames = 'exports', i = ['source', 'dest'], j = 'year', sep = "_")
df2 = df2.reset_index()
print(df2)
# unpivot
df3 = pd.pivot_table(df2, index = ['source', 'dest'], columns = 'year', values = 'exports')
df3.columns = ['exports_' + str(gdpcol) for gdpcol in df3.columns]
df3 = df3.reset_index()
print(df3)

  source dest  exports_2000  exports_2001
0    AUS  AUS             1             2
1    CAN  AUS             2             3
2    AUS  CAN             3             4
3    CAN  CAN             4             5
  source dest  year  exports
0    AUS  AUS  2000        1
1    AUS  AUS  2001        2
2    CAN  AUS  2000        2
3    CAN  AUS  2001        3
4    AUS  CAN  2000        3
5    AUS  CAN  2001        4
6    CAN  CAN  2000        4
7    CAN  CAN  2001        5
  source dest  exports_2000  exports_2001
0    AUS  AUS             1             2
1    AUS  CAN             3             4
2    CAN  AUS             2             3
3    CAN  CAN             4             5


## Pivot - Multiple Index/Multiple Values

In [8]:
# i is the index in the wide version
# j is the name of the second index created from suffixes in the long version
# stubnames are the variables that are being indexed i.e. the first part of the variables that are currently given across the second index
df = pd.DataFrame({'source': ['AUS', 'CAN', 'AUS', 'CAN'], 'dest': ['AUS', 'AUS', 'CAN', 'CAN'], 'exports_2000': [1, 2, 3, 4], 'exports_2001': [2, 3, 4, 5], 'imports_2000': [1, 2, 3, 4], 'imports_2001': [2, 3, 4, 5]})
print(df)
df2 = pd.wide_to_long(df, stubnames = ['imports', 'exports'], i = ['source', 'dest'], j = 'year', sep = "_")
df2 = df2.reset_index()
print(df2)
# unpivot
df3 = pd.pivot_table(df2, index = ['source', 'dest'], columns = 'year', values = ['exports', 'imports'])
# with multiple values, get multi-index column that need to combine manually
df3.columns = [col[0] + '_' + str(col[1]) for col in df3.columns]
df3 = df3.reset_index()
print(df3)

  source dest  exports_2000  exports_2001  imports_2000  imports_2001
0    AUS  AUS             1             2             1             2
1    CAN  AUS             2             3             2             3
2    AUS  CAN             3             4             3             4
3    CAN  CAN             4             5             4             5
  source dest  year  imports  exports
0    AUS  AUS  2000        1        1
1    AUS  AUS  2001        2        2
2    CAN  AUS  2000        2        2
3    CAN  AUS  2001        3        3
4    AUS  CAN  2000        3        3
5    AUS  CAN  2001        4        4
6    CAN  CAN  2000        4        4
7    CAN  CAN  2001        5        5
  source dest  exports_2000  exports_2001  imports_2000  imports_2001
0    AUS  AUS             1             2             1             2
1    AUS  CAN             3             4             3             4
2    CAN  AUS             2             3             2             3
3    CAN  CAN             4 

## CPI Example

In [16]:
# the main object index is name
# other object index is levelid
# the main date is month
# Two variables to reshape are cpi_nsa, cpi_sa
df = pd.DataFrame({'name': ['All items', 'All items', 'Food', 'Food', 'Energy', 'Energy', 'All items except food and energy', 'All items except food and energy'], 'month': ['202101m', '202102m', '202101m', '202102m', '202101m', '202102m', '202101m', '202102m'], 'levelid': ['00', '00', '00_00', '00_00', '00_01', '00_01', '00_02', '00_02'], 'cpi_nsa': [100, 101, 200, 202, 300, 303, 400, 404], 'cpi_sa': [100, 100, 200, 200, 300, 300, 400, 400]})
print(df)

# long to wide
df2 = pd.pivot_table(df, index = ['name', 'levelid'], columns = 'month', values = ['cpi_sa', 'cpi_nsa'])
# adjust columns
df2.columns = [col[0] + '_' + col[1] for col in df2.columns]
df2 = df2.reset_index()

print(df2)

# wide to long
# w+ matches [a-zA-Z0-9_]+
stubnames = sorted(list(set(['_'.join(col.split('_')[: -1]) for col in df2.columns if col.startswith('cpi_nsa_') or col.startswith('cpi_sa_')])))
df3 = pd.wide_to_long(df2, stubnames = ['cpi_nsa', 'cpi_sa'], i = ['name', 'levelid'], j = 'month', sep = "_", suffix = '\\w+')
df3 = df3.reset_index()
print(df3)

# alternatively reshaping long to wide with name_cpi_nsa rather than date_cpi_nsa as column
df4 = pd.pivot_table(df3, index = ['month', 'levelid'], columns = 'name', values = ['cpi_sa', 'cpi_nsa'])
# adjust columns
df4.columns = [col[1] + '_' + col[0] for col in df4.columns]
df4 = df4.reset_index()
print(df4)

                               name    month levelid  cpi_nsa  cpi_sa
0                         All items  202101m      00      100     100
1                         All items  202102m      00      101     100
2                              Food  202101m   00_00      200     200
3                              Food  202102m   00_00      202     200
4                            Energy  202101m   00_01      300     300
5                            Energy  202102m   00_01      303     300
6  All items except food and energy  202101m   00_02      400     400
7  All items except food and energy  202102m   00_02      404     400
                               name levelid  cpi_nsa_202101m  cpi_nsa_202102m  \
0                         All items      00              100              101   
1  All items except food and energy   00_02              400              404   
2                            Energy   00_01              300              303   
3                              Food   00_00   